Submitting various things for end of grant.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import ENCODED, DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [4]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~diane/woldlab/ENCODE/stranded-20200813.xlsx')

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [5]:
award = 'UM1HG009443'

# Get biosample ontologies

In [6]:
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0)
created = 0

for i, row in biosamples.iterrows():
    if pandas.isnull(row['biosample_ontology']):
        accession = row['accession']
        biosample = server.get_json(accession)
        #print(accession, biosample['biosample_ontology']['@id'])
        biosamples.loc[i, 'biosample_ontology'] = biosample['biosample_ontology']['@id']
        created += 1
    if row['source'][0] != '/':
        biosample = server.get_json(accession)
        biosamples.loc[i, 'source'] = biosample['source']['@id']
        created += 1

if created > 0:
    biosamples.to_excel('/dev/shm/biosamples.xlsx')
#biosamples
created

0

In [7]:
biosamples

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,library_id:skip,cDNA_sample:skip,organism,source,lab,award
0,NaN,ENCBS713BMM,NaN,/biosample-types/cell_line_EFO_0003045/,NaN,H9 ESC,22537,ENC4_cDNA227,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443
1,NaN,ENCBS654ADT,NaN,/biosample-types/cell_line_EFO_0003045/,NaN,H9 ESC,22538,ENC4_cDNA228,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443
2,NaN,ENCBS601OTT,NaN,/biosample-types/in_vitro_differentiated_cells...,NaN,H9 derived definitive endoderm,22539,ENC4_cDNA229,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443
3,NaN,ENCBS958FWY,NaN,/biosample-types/in_vitro_differentiated_cells...,NaN,H9 derived definitive endoderm,22540,ENC4_cDNA230,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443
4,NaN,ENCBS832MSN,NaN,/biosample-types/in_vitro_differentiated_cells...,NaN,H9 derived pancreatic progenitor cells,22541,ENC4_cDNA231,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443
5,NaN,ENCBS825BIV,NaN,/biosample-types/in_vitro_differentiated_cells...,NaN,H9 derived pancreatic progenitor cells,22542,ENC4_cDNA232,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443
6,NaN,ENCBS779FCS,NaN,/biosample-types/in_vitro_differentiated_cells...,NaN,H9 derived insulin producing beta-like cells,22543,ENC4_cDNA233,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443
7,NaN,ENCBS251SEQ,NaN,/biosample-types/in_vitro_differentiated_cells...,NaN,H9 derived insulin producing beta-like cells,22544,ENC4_cDNA234,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443
8,NaN,ENCBS200JOD,NaN,/biosample-types/in_vitro_differentiated_cells...,NaN,H9 derived neural crest cells,22545,ENC4_cDNA235,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443
9,NaN,ENCBS417JEU,NaN,/biosample-types/in_vitro_differentiated_cells...,NaN,H9 derived neural crest cells,22546,ENC4_cDNA236,/organisms/human/,/sources/sescc/,barbara-wold,UM1HG009443


# Submit Documents

Example Document submission

In [8]:
#atac_uuid = '0fc44318-b802-474e-8199-f3b6d708eb6f'
#atac = Document(os.path.expanduser('~/proj/encode3-curation/Wold_Lab_ATAC_Seq_protocol_December_2016.pdf'),
#                'general protocol',
#                'ATAC-Seq experiment protocol for Wold lab',
#                )
#body = atac.create_if_needed(server, atac_uuid)
#print(body['@id'])

# Submit Annotations

In [9]:
#sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#annotations = sheet.parse('Annotations', header=0)
#created = server.post_sheet('/annotations/', annotations, verbose=True, dry_run=True)
#print(len(created))

In [10]:
#if created:
#    annotations.to_excel('/tmp/annotations.xlsx', index=False)

# Register Biosamples

In [11]:
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0)
created = server.post_sheet('/biosamples/', biosamples, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [12]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [21]:
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0)
created = server.post_sheet('/libraries/', libraries, verbose=True, dry_run=True, validator=validator)
print(len(created))

0


In [20]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [24]:
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0)
created = server.post_sheet('/experiments/', experiments, verbose=True, dry_run=True, validator=validator)
print(len(created))

0


In [23]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [29]:
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0)
created = server.post_sheet('/replicates/', replicates, verbose=True, dry_run=True, validator=validator)
print(len(created))

0


In [28]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [32]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

24
